#Library and functions

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from collections import Counter
import PIL
import glob
from tqdm import tqdm
import networkx as nx

#Graphics
from scipy.stats import gaussian_kde
from scipy.stats import norm
import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

#NN
from tensorflow import keras
from tensorflow.keras import layers, callbacks
#import tensorflow as tf

#sklearn modules: evaluation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, classification_report, RocCurveDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Recurrence plots
from pyts.image import RecurrencePlot
from pyts.datasets import load_gunpoint
from mpl_toolkits.axes_grid1 import ImageGrid

/home/llober/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-08-29 15:48:59.969044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 15:49:00.591609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
def parameter_stratification(value):
  '''
  Assumes the parameters of the logistic map will be in the 3 to 4 range.
  Categories:
  1 - Oscillations between two values
  2 - Oscillations between four values
  3 - Oscillations between nth values, where n is even and >4
  4 - Onset of chaos
  5 - Appearance of stability islands
  '''

  #Periods
  if (value < 3.44949):
    return 1
  if ( (value >= 3.44949) and (value < 3.54409) ):
    return 2
  if ( (value >= 3.54409) and (value < 3.56995) ):
    return 3
  if ( (value >= 3.56995) and (value < 3.82843) ):
    return 4
  if ( (value >= 3.82843) ):
    return 5

# Neural net training: regression

In [ ]:
#Read all pngs from the train set
pngs = glob.glob('your_path/IC=0.*/sample*png')

ims = {}
for png in pngs:
  ims[png]=np.array(PIL.Image.open(png).convert('L'))

In [ ]:
#Questions = images
#Solutions = main parameter of the logistic map used to generated that image

questions = np.array([each for each in ims.values()]).astype(np.float32)
solutions = np.array([float(each.split('_')[-1].split('.')[0])/(1e16) for each in ims]).astype(np.float32)

##Architecture

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(units=1024, activation='relu'),
  keras.layers.Dense(units=512, activation='relu'),
  keras.layers.Dense(units=256, activation='relu'),
  keras.layers.Dense(units=64, activation='relu'),
  keras.layers.Dense(units=1)
])

model.compile(loss='mean_squared_error', optimizer="adam")

## Training

In [ ]:
N_size = 100

#Train/test split (gnuplot data):
X_train, X_test, y_train, y_test = train_test_split(questions, solutions, test_size=0.3, random_state=42)

X_train = X_train.reshape(-1, N_size, N_size, 1)
X_test = X_test.reshape(-1, N_size, N_size, 1)

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=100,
    epochs=300,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

##Testing

In [ ]:
#Read in pngs:
test_pngs = glob.glob('your_path/test/sample*png')

test_ims = {}

for png in test_pngs:
  test_ims[png]=np.array(PIL.Image.open(png).convert('L'))


#Prepare test questions and solutions as before:
test_questions = np.array([each for each in test_ims.values()]).astype(np.float32)
test_solutions = np.array([float(each.split('_')[-1].split('.')[0])/(1e16)
                            for each in test_ims]).astype(np.float32)

#Apply model:
test_answers = model.predict(test_questions)

In [ ]:
print('MSE:', mean_squared_error(test_answers, test_solutions))

MSE: 0.001876895


In [ ]:
df_res = pd.DataFrame(data={'test_actual': test_solutions, 'test_predictions': test_answers.flatten(), 'N': np.repeat(100, len(test_answers))})
df_res.to_csv('NN_RP_scatter_reg_n_100_p20_chaos_40k.csv')

# Neural net training: classification

In [ ]:
#Read all pngs from the train set
pngs = glob.glob('your_path/IC=0.*/sample*png')

ims = {}
for png in pngs:
    ims[png]=np.array(PIL.Image.open(png).convert('L'))

In [ ]:
#Questions = images
#Solutions = a parameter that generated that image

questions = np.array([each for each in ims.values()]).astype(np.float32)
solutions_swp = np.array([float(each.split('_')[-1].split('.')[0])/(1e16)
                            for each in ims]).astype(np.float32)

solutions = []
for n in solutions_swp:
  solutions.append(parameter_stratification(n))

##Architecture

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(3, 3),
  keras.layers.Conv2D(3, 3),
  keras.layers.Activation('relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Dropout(0.2),
  keras.layers.Conv2D(3, 3),
  keras.layers.Conv2D(3, 3),
  keras.layers.Activation('relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(units=512, activation='relu'),
  keras.layers.Dense(units=256, activation='relu'),
  keras.layers.Dense(units=64, activation='relu'),
  keras.layers.Dense(units=6, activation='softmax')
])

model.compile(loss='CategoricalCrossentropy', optimizer="adam", metrics=['accuracy'])

## Training

In [ ]:
N_size = 100

#Train/test split (gnuplot data):
X_train, X_test, y_train, y_test = train_test_split(questions, solutions, test_size=0.3, random_state=42)

X_train = X_train.reshape(-1, N_size, N_size, 1)
X_test = X_test.reshape(-1, N_size, N_size, 1)

In [ ]:
# Get one hot encoding
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=100,
    epochs=300,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

##Testing

In [ ]:
test_pngs = glob.glob('your_path/test/sample*png')    #chaotic region

test_ims = {}

for png in test_pngs:
    test_ims[png]=np.array(PIL.Image.open(png).convert('L'))

test_questions = np.array([each for each in test_ims.values()]).astype(np.float32)
test_solutions_swp = np.array([float(each.split('_')[-1].split('.')[0])/(1e16)
                            for each in test_ims]).astype(np.float32)

test_solutions = []
for n in test_solutions_swp:
  test_solutions.append(parameter_stratification(n))

In [ ]:
#Apply model:
test_answers = model.predict(test_questions)

test_answers_classes = [np.argmax(y, axis=None, out=None) for y in test_answers]
print('Accuracy:', accuracy_score(test_answers_classes, test_solutions))

In [ ]:
#le = LabelEncoder()
#test_y_swp = le.fit_transform(test_y)

label_binarizer = LabelBinarizer().fit(test_solutions)
test_y_swp = label_binarizer.transform(test_solutions)
pred_y_swp = label_binarizer.transform(test_answers_classes)

In [ ]:
report = classification_report(test_solutions, test_answers_classes, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report